<a href="https://colab.research.google.com/github/EmanueleGiavardi/AMD_project/blob/main/src/amd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
import json
from collections import Counter

In [2]:
# kaggle authentication and dataset download

drive.mount("/content/gdrive", force_remount=True)

# PLEASE PROVIDE HERE A VALID PATH FOR kaggle.json FILE, containing username and key for Kaggle
KAGGLE_FILE_PATH = "/content/gdrive/MyDrive/.kaggle/kaggle.json"
kaggle_json = json.load(open(KAGGLE_FILE_PATH))

os.environ['KAGGLE_USERNAME'] = kaggle_json['username']
os.environ['KAGGLE_KEY'] = kaggle_json['key']
!kaggle datasets download -d "mohamedbakhet/amazon-books-reviews"
!unzip amazon-books-reviews.zip
!rm -r amazon-books-reviews.zip

Mounted at /content/gdrive
Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
100% 1.06G/1.06G [00:07<00:00, 271MB/s]
100% 1.06G/1.06G [00:07<00:00, 158MB/s]
Archive:  amazon-books-reviews.zip
  inflating: Books_rating.csv        
  inflating: books_data.csv          


In [3]:
books_rating_df = pd.read_csv("Books_rating.csv", sep=",")
books_data_df = pd.read_csv("books_data.csv", sep=",")

In [75]:
# subsampling

random_state = 42
# randomly sample the sampling_frac*100% of the original rating dataset
sampling_frac = 0.1

books_rating_df_sub = books_rating_df.sample(frac=sampling_frac, random_state=random_state).reset_index(drop=True)

In [77]:
unique_book_titles = Counter(books_rating_df_sub['Title']).keys()
print(f"subsampled rating dataframe has {len(unique_book_titles)} unique book titles")

# keeping data only for books in subsampled rating dataframe
books_data_df_sub = books_data_df[books_data_df['Title'].isin(unique_book_titles)]
print(f"subsampled book dataframe len: {len(books_data_df_sub)}")

subsampled rating dataframe has 78452 unique book titles
subsampled book dataframe len: 78452


In [78]:
# maybe performing the merge is not the optimal thing, due to data replication. I could simply access the data I need using 'Title' as index
# this depends on the actual task to perform

joint_books_df = pd.merge(books_rating_df_sub, books_data_df, left_on='Title', right_on='Title', how='left')
joint_books_df

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,B0006CR6U4,"A dictionary of the Targumim, the Talmud Babli...",NaN,A303XPDO694V6X,Ariel,2/6,4.0,1122163200,Jastrow,Jastrow made a great workthis dictionary can h...,"New, larger edition! Commonly known as the Jas...",['Marcus Jastrow'],http://books.google.com/books/content?id=S4UYA...,http://books.google.com/books?id=S4UYAQAAIAAJ&...,Judaica Press,2004,http://books.google.com/books?id=S4UYAQAAIAAJ&...,['Reference'],NaN
1,0897166159,Espresso Coffee: Professional Techniques,NaN,A3780H4TM9RMB8,David barnes,0/1,2.0,1356912000,NOT the book,Extremely disappointed by the SHORT length and...,When I started a coffee company in April of 19...,['David C. Schomer'],NaN,http://books.google.com/books?id=uEWUAAAACAAJ&...,NaN,2004-01-01,http://books.google.com/books?id=uEWUAAAACAAJ&...,['Coffee'],NaN
2,0736693408,The First King of Shannara (The Sword of Shann...,NaN,A1AX6VPDQQZDPV,M Carlton,4/4,5.0,1105574400,Great (what do you expect?),"This, like all of Brook's Shannara series book...",BONUS: This edition contains an excerpt from T...,['Terry Brooks'],http://books.google.com/books/content?id=eBq_s...,http://books.google.com/books?id=eBq_sufonjwC&...,Del Rey,2000-10-10,https://play.google.com/store/books/details?id...,['Fiction'],91.0
3,0395051029,Wuthering Heights (Riverside editions),NaN,A35RQKCCCQ62O0,LadyJ,0/0,4.0,1353888000,Satisfied,I enjoyed this classic. I didn't know the stor...,NaN,['Emily Brontë'],NaN,http://books.google.com/books?id=VSfNtwEACAAJ&...,NaN,1963,http://books.google.com/books?id=VSfNtwEACAAJ&...,NaN,NaN
4,4770016050,"A Cat, a Man, and Two Women (Japans Modern Wri...",NaN,A2IJQDE1I4SIJT,"David C. Arnold ""master D""",1/2,5.0,1167955200,"Ordered 09/02/2006, still on backorder",I would love to read this book. Have accepted ...,A novella and two short stories reveal Tanizak...,"['Junichiro Tanizaki', 'Paul McCarthy']",http://books.google.com/books/content?id=3bPBo...,http://books.google.com/books?id=3bPBoQEACAAJ&...,New Directions,2015-08-06,http://books.google.com/books?id=3bPBoQEACAAJ&...,['Fiction'],1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,B0007I5RRG,"One fish, two fish, red fish, blue fish (I can...",NaN,NaN,NaN,4/6,5.0,1083456000,An astonishing adventure into human philosophy,"An epic, amazing, astonishing tale into true p...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299996,0131018612,"Principles of Marketing, 10th Edition",NaN,A9F09NE3YSY5J,"Terry Phan ""Terry""",0/5,4.0,1127520000,Book,The book was in good condition but it was not ...,Dit is de vierde editie van Principes van mark...,"['Philip J. Kotler', 'Ph. Kotler G. Armstrong']",http://books.google.com/books/content?id=MwbT5...,http://books.google.nl/books?id=MwbT5MpdUS4C&p...,Pearson Education,2005,http://books.google.nl/books?id=MwbT5MpdUS4C&d...,NaN,4.0
299997,1591604443,The Revival We Need,NaN,A1S4R57Z57YRFE,"Glenn H. Mcnease ""revelation 11 witness""",0/0,5.0,1281225600,XCELLENTA PERFECTO PRIMO,THIS IS ONE OF A VERY FEW BOOKS EVER WRITTEN P...,When Oswald J. Smith wrote this book almost a ...,['Oswald Smith'],http://books.google.com/books/content?id=-YVtM...,http://books.google.nl/books?id=-YVtMAEACAAJ&d...,Kingsley Press,2012-06-28,http://books.google.nl/books?id=-YVtMAEACAAJ&d...,NaN,NaN
299998,0753816504,John Donne: Selected Poems (Phoenix Poetry),NaN,AHD101501WCN1,"Shalom Freedman ""Shalom Freedman""",2/2,5.0,1268438400,The poetry of Argument and Intellectual Feeling,This book's editor Andrews Wanning provides a ...,NaN,"['John Donne', 'D. J. Enright']",NaN,http://books.google.com/books?id=k8iZZwEACAAJ&...,NaN,2010,http://books.google.com/books?id=k8iZZwEACAAJ&...,['English poetry'],NaN
